In [89]:
import pandas as pd
import datetime as dt
import matplotlib as m
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from collections import OrderedDict
import operator
import Heavy_Cash_Withdrawal_in_a_month as heavyCashWithdrawal
import Heavy_Cash_Deposit_in_a_month as heavyCashDeposit
import json
import requests

In [90]:
#loading paysim and berka dataset
berka_dataset=pd.read_csv("dataset.csv")
paysim_dataset = pd.read_csv("paysim.csv")
paysim_dataset=paysim_dataset.dropna(how='any')

In [91]:
def analyseNetworkGraph(df):
    # Network Analysis taking 10 nodes
    node_x=df[['nameOrig','nameDest','amount']][1:10]
    H = nx.from_pandas_edgelist(node_x, 'nameOrig', 'nameDest','amount')
    G = nx.DiGraph()
    G.add_edges_from(H.edges())
    '''pos = nx.random_layout(G)
    nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
                        node_size = 500)
    nx.draw_networkx_labels(G, pos)
    nx.draw(G,pos,node_size=100,font_size=100)
    nx.draw_networkx_edges(G, pos,  edge_color='r', arrows=True)
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(),edge_color='b',arrows=True)
    plt.figure(3,figsize=(100,100)) 
    plt.show()'''

In [92]:
def pageRank(G,node_x):
    pr=nx.pagerank(G,0.85)
    descending = sorted_x = sorted(pr.items(), key=operator.itemgetter(1),reverse=True)
    Indegree=G.in_degree()
    node_x=np.array(node_x)
    for i in range(len(node_x)):
        if(Indegree[node_x[i][1]] == 0 or Indegree[node_x[i][1]] ==1 ):
            #print('Hii')
            k=-1
            for j in descending :
                k+=1
               # print(j[0]," ",node_x[i][1])
                if( j[0]==node_x[i][1]):
                    descending.pop(k)
    return descending

In [93]:
def hitsAlgo(G,node_x):
    hubs,a=nx.hits(G,max_iter=100,tol=1e-08,nstart=None)
    hubs_list = sorted_x = sorted(a.items(), key=operator.itemgetter(1),reverse=True)
    Indegree=G.in_degree()
    node_x=np.array(node_x)
    for i in range(len(node_x)):
        if(Indegree[node_x[i][1]] == 0 or Indegree[node_x[i][1]] ==1 ):
            #print('Hii')
            k=-1
            for j in hubs_list :
                k+=1
               # print(j[0]," ",node_x[i][1])
                if( j[0]==node_x[i][1]):
                    hubs_list.pop(k)
    return hubs_list,a

In [94]:
analyseNetworkGraph(paysim_dataset)
df=paysim_dataset[['nameOrig','nameDest','amount']][1:1000]
H = nx.from_pandas_edgelist(df, 'nameOrig', 'nameDest','amount')
G = nx.DiGraph()
G.add_edges_from(H.edges())
mostSuspiciousAccounts=pageRank(G,df)
#Generating Suspicious Transaction along with their score using Hits Algorithm
hubs_list,authority_list = hitsAlgo(G,df)
#for i in hubs_list:
#    print ("Account id: ",i[0] ," ",i[1])

In [95]:
paysim_arr=np.array(paysim_dataset)
thresold=0.2
suspiciousTransactions=[]
for i in hubs_list:
    for index in range(len(paysim_arr)):
        if(i[1]<thresold):
            break
        if(i[0]==paysim_arr[index][6]):
                #print(paysim_arr[index][6])
                suspiciousTransactions.append(paysim_arr[index])

In [96]:
berka_dataset['operation'] = berka_dataset['operation'].replace(['CASH IN'], 0)
berka_dataset['operation'] = berka_dataset['operation'].replace(['CASH WITHDRAWAL'], 1)
berka_dataset['operation'] = berka_dataset['operation'].replace(['BANK TRANSFER'], 2)
maxAccID = berka_dataset['account_id'][len(berka_dataset)-1]
accountlist=set(berka_dataset['account_id'])

In [97]:
cashWithdrawalList = heavyCashWithdrawal.heavyWithdrawal(berka_dataset,accountlist,maxAccID)
cashDepositList = heavyCashDeposit.heavyDeposit(berka_dataset,accountlist,maxAccID)


# Sample json 
{
  "$class": "org.acme.model.suspiciousAccountAsset",
  "Kycid": "321",
  "details": [
    {
      "$class": "org.acme.model.suspiciousAccountDetails",
      "accountId": "1",
      "custBank": "resource:org.acme.model.Bank#123",
      "transArr": [
        {
          "$class": "org.acme.model.suspiciousTransaction",
          "otherCustBank": "resource:org.acme.model.Bank#123",
          "other_acc_id": "576",
          "amount": 12,
          "modeOfPayment": "transfer",
          "Date": "05-05-05",
          "id": "string"
        }
      ],
      "suspiciousScore": 0.8,
      "id": "string"
    }
  ]
}

In [98]:
suspiciousAccount="org.acme.model.suspiciousAccountAsset"
susAcc="org.acme.model.suspiciousAccountDetails"
susTrans="org.acme.model.suspiciousTransaction"
#cashWithdrawalList

In [99]:
tempStr = 'KYCID'
mappingList=[]
for index in range(1,103):
    if(index<10):
        kycid =  tempStr + '00' + str(index)
    elif(index<100):
        kycid =  tempStr + '0' + str(index)
    else:
        kycid =  tempStr + str(index)
    mappingList.append([index,kycid])
mappingList[0][1]

'KYCID001'

In [100]:
suspiciousTransactionList= cashDepositList + cashWithdrawalList 
len(suspiciousTransactionList)

960

In [101]:
sortedList=sorted(suspiciousTransactionList, key=lambda x: x[1])

In [102]:
transArr={"$class": susTrans, "otherCustBank" :"resource:org.acme.model.Bank#123",
          "other_acc_id":"","amount":cashWithdrawalList[0][4],"modeOfPayment":cashWithdrawalList[0][7],
         "Date":cashWithdrawalList[0][8]}
trans_list=[]
trans_list.append(transArr)

In [103]:
trans_list=[]
acc_id=sortedList[0][1]
for i in range(len(sortedList)):
    transArr={"$class": susTrans, "otherCustBank" :"resource:org.acme.model.Bank#123",
          "other_acc_id":"","amount":sortedList[i][4],"modeOfPayment":sortedList[i][7],
         "Date":sortedList[i][8]}
    trans_list.append(transArr)
    if((i+1<len(sortedList) and acc_id != sortedList[i+1][1]) or (i==len(sortedList)-1)):
        details={"$class": susAcc,"accountId":sortedList[i][1],"custBank":"resource:org.acme.model.Bank#123",
        "transArr":trans_list,"suspiciousScore": 0.8}
        details_list=[]
        details_list.append(details)
        output={"$class": suspiciousAccount, "Kycid":mappingList[sortedList[i][1]-1][1],"details":details_list}
        d=json.dumps(output)
        d=json.loads(d)
        print(d,"\n")
        url="http://localhost:3000/api/org.acme.model.suspiciousAccountAsset"
        response = requests.post(url,json=d)
        trans_list=[]
        if(i+1<len(sortedList)):
            acc_id=sortedList[i+1][1]

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID001', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 1, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 12600.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '04/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 12600.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '04/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 12600.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '04/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}], 'suspiciousScore': 0.8}]} 

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID009', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 9, '$class': 'org.acme.model.suspiciousAccou

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID018', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 18, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 7706.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '03/94', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 44907.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '03/94', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 35017.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '03/94', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 21900.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '07/97', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 3

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID023', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 23, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 10800.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 33132.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 10800.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 33132.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID031', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 31, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 30311.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 6113.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 28892.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 28587.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 4

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID040', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 40, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 6502.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '11/93', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 8400.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '11/93', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 6502.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '11/93', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 8400.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '11/93', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 6502

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID070', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 70, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 4320.0, 'modeOfPayment': 'withdraw', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '12/97', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 14.6, 'modeOfPayment': 'withdraw', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '12/97', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 4320.0, 'modeOfPayment': 'withdraw', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '12/97', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 14.6, 'modeOfPayment': 'withdraw', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '12/97', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID083', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 83, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 9226.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 7900.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '01/96', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 10100.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '02/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 9226.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '02/95', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 350

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID100', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 100, '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [{'other_acc_id': '', 'amount': 16100.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '02/98', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 11891.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '02/98', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 17836.5, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/94', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 6800.0, 'modeOfPayment': 'credit', '$class': 'org.acme.model.suspiciousTransaction', 'Date': '06/94', 'otherCustBank': 'resource:org.acme.model.Bank#123'}, {'other_acc_id': '', 'amount': 

In [104]:
df=paysim_dataset['nameDest']
account=set(df)
len(df)
df=list(account)

In [105]:
tempStr = 'KYCID'
mappingList={}
for index in range(len(df)):
    if(index<10):
        kycid =  tempStr + '000' + str(index)
    elif(index<100):
        kycid =  tempStr + '00' + str(index)
    else:
        kycid =  tempStr + str(index)
    mappingList[df[index]]=kycid
    #rint(mappingList[df[i]])


In [106]:
#pay_sim_account=list(set(suspiciousTransactions))
trans_list=[]
#acc_id=pay_sim_account[0][1]
for i in hubs_list:
    '''transArr={"$class": susTrans, "otherCustBank" :"resource:org.acme.model.Bank#123",
          "other_acc_id":"","amount":sortedList[i][4],"modeOfPayment":sortedList[i][7],
         "Date":sortedList[i][8]}
    trans_list.append(transArr)'''
    if(i[1]>0.2):
        details={"$class": susAcc,"accountId":i[0],"custBank":"resource:org.acme.model.Bank#123",
        "transArr":trans_list,"suspiciousScore": round(i[1],3)}
        details_list=[]
        details_list.append(details)
        output={"$class": suspiciousAccount, "Kycid":mappingList[i[0]],"details":details_list}
        d=json.dumps(output)
        d=json.loads(d)
        print(d,"\n")
        url="http://localhost:3000/api/org.acme.model.suspiciousAccountAsset"
        response = requests.post(url,json=d)
        trans_list=[]
    else:
        break

{'$class': 'org.acme.model.suspiciousAccountAsset', 'Kycid': 'KYCID49037', 'details': [{'custBank': 'resource:org.acme.model.Bank#123', 'accountId': 'C985934102', '$class': 'org.acme.model.suspiciousAccountDetails', 'transArr': [], 'suspiciousScore': 0.5}]} 



In [107]:
transArr={"$class": susTrans, "otherCustBank" :"resource:org.acme.model.Bank#123",
          "other_acc_id":"","amount":cashWithdrawalList[i][4],"modeOfPayment":cashWithdrawalList[i][7],
         "Date":cashWithdrawalList[i][8]}
trans_list.append(transArr)

TypeError: list indices must be integers or slices, not tuple

In [ ]:
details={"$class": susAcc,"accountId":cashWithdrawalList[i][3],"custBank":"resource:org.acme.model.Bank#123",
        "transArr":trans_list,"suspiciousScore": 0.8}
details_list=[]
details_list.append(details)

In [ ]:
output={"$class": suspiciousAccount, "Kycid":mappingList[cashWithdrawalList[i][3]-1][1],"details":details_list}
d=json.dumps(output)
d=json.loads(d)
print(d)

In [ ]:
url="http://localhost:3000/api/org.acme.model.suspiciousAccountAsset"
response = requests.post(url,json=d)

In [ ]:
response.text

In [ ]:
mappingList[cashWithdrawalList[4][3]][1]

In [ ]:
cashWithdrawalList[3][1]

In [ ]:
details={"$class": susAcc,"accountId":cashWithdrawalList[0][3],"custBank":"resource:org.acme.model.Bank#123",
        "transArr":trans_list,"suspiciousScore": 0.8}

In [ ]:
details

In [ ]:
details_list=[]
details_list.append(details)

In [ ]:
details_list

In [ ]:
output={"$class": suspiciousAccount, "Kycid":"KYCID0001","details":details_list}

In [ ]:
d=json.dumps(output)
d=json.loads(d)
print(d)
url="http://localhost:3000/api/org.acme.model.suspiciousAccountAsset"
response = requests.post(url,json=d)
response.text

In [ ]:
import json
import requests

In [ ]:
cashWithdrawalList[0][4]

In [ ]:
transArr={"$class": susTrans, "otherCustBank" :"resource:org.acme.model.Bank#123",
          "other_acc_id":"","amount":cashWithdrawalList[0][4],"modeOfPayment":cashWithdrawalList[i][7],
         "Date":cashWithdrawalList[i][8]}
trans_list.append(transArr)
cash

In [ ]:
details={"$class": susAcc,"accountId":cashWithdrawalList[i][3],"custBank":"resource:org.acme.model.Bank#123",
        "transArr":trans_list,"suspiciousScore": 0.8}
details_list=[]
details_list.append(details)

In [ ]:
output={"$class": suspiciousAccount, "Kycid":mappingList[cashWithdrawalList[i][3]-1][1],"details":details_list}
d=json.dumps(output)
d=json.loads(d)
print(d)


In [ ]:
url="http://localhost:3000/api/org.acme.model.suspiciousAccountAsset"
response = requests.post(url,json=d)

In [ ]:
response.status_code
